<h1 align=center><font size = 5>This is a Jupyter Notebook to import neighborhoods data for Toronto from Wikipedia page.</font></h1>

## 1. Let's first import all libraries.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium

## 2. Now we will scrape the data for Toronto from Wikipedia using pandas. We have also removed the 'Borough' entries which have a value of 'not assigned'.

In [94]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = pd.read_html(url)
toronto_df = pd.DataFrame(toronto_data[0])
toronto_df = toronto_df[toronto_df['Borough']!='Not assigned'].reset_index()
toronto_df.drop('index',axis=1,inplace=True)

## 3. Display the dataframe

In [95]:
toronto_df.head(30)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## 4. Verify if we have 'Not assigned' in column 'Neighborhood'.

In [89]:
print('There are {} "Not Assigned" entries in "Neighborhood" column'.format(len(toronto_df[toronto_df['Neighborhood']=='Not assigned'])))

There are 0 "Not Assigned" entries in "Neighborhood" column


## 5. Display the shape of Toronto DataFrame

In [91]:
print(toronto_df.shape)
print('There are {} rows and {} columns in Toronto DataFrame.'.format(toronto_df.shape[0],toronto_df.shape[1]))

(103, 3)
There are 103 rows and 3 columns in Toronto DataFrame.


## 6. Get Latitude and Longitude for each neighborhood.

In [55]:
location_data = pd.read_csv('http://cocl.us/Geospatial_data')

In [85]:
location_data.head(30)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [98]:
toronto_merged = toronto_df
toronto_merged = toronto_merged.join(location_data.set_index('Postal Code'),on='Postal Code')
toronto_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
